# AOG Text Generation Example

This notebook demonstrates how to use the AOG Chat API for text generation and conversation.

## Table of Contents
1. [Environment Setup](#1-Environment-Setup)
2. [Helper Functions](#2-Helper-Functions)
3. [Basic Chat Example](#3-Basic-Chat-Example)
4. [Different Message Types](#4-Different-Message-Types)
5. [Multi-turn Conversation](#5-Multi-turn-Conversation)
6. [Parameter Configuration](#6-Parameter-Configuration)

## 1. Environment Setup

First, import necessary libraries and configure AOG service connection parameters.

In [ ]:
# Import necessary libraries
import requests
import json
from typing import List, Dict, Optional

# AOG service configuration
AOG_BASE_URL = "http://localhost:16688"  # Modify this if service is at a different address
AOG_API_SPEC_VERSION = "v0.2"
DEFAULT_MODEL_CHAT = "qwen2.5:0.5b"  # Default chat model

print("✅ Configuration complete")
print(f"📍 AOG service address: {AOG_BASE_URL}")
print(f"🤖 Default model: {DEFAULT_MODEL_CHAT}")

## 2. Helper Functions

Define helper functions to call the AOG Chat API with complete error handling.

In [ ]:
def call_aog_chat(
    messages: List[Dict[str, str]], 
    model: str = DEFAULT_MODEL_CHAT, 
    stream: bool = False
) -> Optional[Dict]:
    """
    Call AOG Chat API
    
    Args:
        messages: List of messages, each containing role and content
        model: Model name to use
        stream: Whether to use streaming response
    
    Returns:
        JSON data from API response, or None if failed
    """
    try:
        # Construct API endpoint URL
        url = f"{AOG_BASE_URL}/aog/{AOG_API_SPEC_VERSION}/services/chat"
        
        # Construct request payload
        payload = {
            "model": model,
            "messages": messages,
            "stream": stream
        }
        
        # Send POST request
        response = requests.post(url, json=payload, timeout=30)
        
        # Check response status
        response.raise_for_status()
        
        # Return JSON response
        return response.json()
        
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to AOG service, please ensure service is running")
        print(f"   Check service address: {AOG_BASE_URL}")
        return None
        
    except requests.exceptions.Timeout:
        print("❌ Request timeout, please check network connection or increase timeout")
        return None
        
    except requests.exceptions.HTTPError as e:
        print(f"❌ API call failed: {e}")
        print(f"   Response content: {response.text}")
        return None
        
    except Exception as e:
        print(f"❌ Unknown error: {e}")
        return None


def print_response(response: Optional[Dict]) -> None:
    """
    Format and print API response
    
    Args:
        response: API response data
    """
    if response is None:
        print("⚠️  No response received")
        return
    
    # Extract assistant's reply
    if "choices" in response and len(response["choices"]) > 0:
        message = response["choices"][0].get("message", {})
        content = message.get("content", "")
        
        print("\n" + "="*60)
        print("🤖 Assistant reply:")
        print("="*60)
        print(content)
        print("="*60 + "\n")
    else:
        print("⚠️  Abnormal response format")
        print(json.dumps(response, indent=2, ensure_ascii=False))

print("✅ Helper functions defined")

## 3. Basic Chat Example

Simplest chat example: send a user message and get an AI reply.

In [ ]:
# Construct a simple user message
messages = [
    {"role": "user", "content": "Hello! Please introduce yourself."}
]

print("📤 Sending message...")
response = call_aog_chat(messages)
print_response(response)

## 4. Different Message Types

Demonstrate how to use different message types: system, user, assistant.

### 4.1 Using System Message to Set Role

In [ ]:
# Use system message to define AI assistant's role
messages = [
    {"role": "system", "content": "You are a professional Python programming assistant, skilled at explaining code and providing programming advice."},
    {"role": "user", "content": "How do I read a JSON file in Python?"}
]

print("📤 Sending message (with system role setting)...")
response = call_aog_chat(messages)
print_response(response)

### 4.2 Different System Prompt Effects

In [ ]:
# Try different system prompts
messages = [
    {"role": "system", "content": "You are a humorous assistant who likes to use metaphors and examples to explain problems."},
    {"role": "user", "content": "What is machine learning?"}
]

print("📤 Sending message (humorous style)...")
response = call_aog_chat(messages)
print_response(response)

## 5. Multi-turn Conversation

Implement contextually coherent multi-turn conversation by maintaining conversation history.

In [ ]:
# Initialize conversation history
conversation = [
    {"role": "system", "content": "You are a friendly assistant."},
    {"role": "user", "content": "I want to learn Python, where should I start?"}
]

print("🗣️  First round of conversation")
print("User: I want to learn Python, where should I start?")
response = call_aog_chat(conversation)
print_response(response)

# Add assistant's reply to conversation history
if response and "choices" in response:
    assistant_message = response["choices"][0]["message"]
    conversation.append(assistant_message)

In [ ]:
# Continue conversation - second round
conversation.append({
    "role": "user", 
    "content": "Do I need to install any software?"
})

print("🗣️  Second round of conversation")
print("User: Do I need to install any software?")
response = call_aog_chat(conversation)
print_response(response)

# Continue adding to conversation history
if response and "choices" in response:
    assistant_message = response["choices"][0]["message"]
    conversation.append(assistant_message)

In [ ]:
# View complete conversation history
print("\n📜 Complete conversation history:")
print("="*60)
for i, msg in enumerate(conversation, 1):
    role = msg["role"]
    content = msg["content"]
    print(f"\n{i}. [{role.upper()}]")
    print(content[:200] + "..." if len(content) > 200 else content)
print("\n" + "="*60)

## 6. Parameter Configuration

Demonstrate how to configure different parameters.

### 6.1 Using Different Models

In [ ]:
# Use specified model
messages = [
    {"role": "user", "content": "Explain what artificial intelligence is in one sentence."}
]

print(f"📤 Using model: {DEFAULT_MODEL_CHAT}")
response = call_aog_chat(messages, model=DEFAULT_MODEL_CHAT)
print_response(response)

### 6.2 Streaming vs Non-streaming Response

Note: This example uses non-streaming response (`stream=False`), which waits for the complete result before returning.
Streaming response (`stream=True`) returns results word by word, suitable for scenarios requiring real-time display.

In [ ]:
# Non-streaming response (default)
messages = [
    {"role": "user", "content": "Write a short poem about spring."}
]

print("📤 Using non-streaming response (stream=False)")
response = call_aog_chat(messages, stream=False)
print_response(response)

## 🎉 Summary

Through this notebook, you have learned:

1. ✅ Configure AOG service connection
2. ✅ Call Chat API for basic conversation
3. ✅ Use different message types (system, user, assistant)
4. ✅ Implement multi-turn conversation with context
5. ✅ Configure model parameters
6. ✅ Handle API errors and exceptions

### Next Steps

- Try different system prompts to explore AI's different roles
- Implement more complex conversation logic
- Check out [Text-to-Image Example](../text-to-image/text-to-image.ipynb)
- Read [AOG API Documentation](../../docs/) for more features

**Happy coding!** 🚀